In [1]:
%matplotlib inline

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv
import time

In [3]:
nb_classes = 2


def print_confusion_matric(nb_classes, dataloader):
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(dataloader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs.float())
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    print(confusion_matrix)
    new_confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i, row in enumerate(confusion_matrix):
        sum = row.sum()
        new_confusion_matrix[i, 0] = 100 * row[0] / sum
        new_confusion_matrix[i, 1] = 100 * row[1] / sum
    conf_matrix = pd.DataFrame(
        new_confusion_matrix.numpy(),
        index=["open", "closed"],
        columns=["open", "closed"],
    )
    # cf.style.background_gradient(cmap='Blues').format("{:.1f} %")
    print("Percent Correct")
    return conf_matrix.style.background_gradient(cmap="Blues").format("{:.1f}")
    # print(confusion_matrix)

In [4]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"
fp_csv_file = "EEG_Eye_State_Train.csv"
# fp_csv_file = "Dummy_Train.csv"
eeg_data = pd.read_csv(
    os.path.join(root_directory, fp_csv_file), delimiter=",", usecols=range(15)
).dropna(axis=0)

# print(eeg_data.iloc[0, :])
open_or_closed = eeg_data.iloc[0, 14]
channel_data = eeg_data.iloc[0, 0:14]
channel_data = np.array([channel_data])
channel_data = channel_data.astype("float").reshape(-1, 14)
# print(open_or_closed)
# print(channel_data)
eeg_data.shape

(11984, 15)

In [5]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Train.csv"
validate_file = "EEG_Eye_State_Validate.csv"
test_file = "EEG_Eye_State_Test.csv"

# train_file = "Dummy_Train.csv"
# validate_file = "Dummy_Validate.csv"
# test_file = "Dummy_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [7]:
# class ConvNet(torch.nn.Module):
#     def __init__(
#         self,
#         n_inputs,
#         n_hiddens_per_conv_layer,
#         n_hiddens_per_fc_layer,
#         n_outputs,
#         patch_size_per_conv_layer,
#         stride_per_conv_layer,
#         activation_function="tanh",
#         device="cpu",
#     ):

#         super().__init__()

#         self.device = device

#         n_conv_layers = len(n_hiddens_per_conv_layer)
#         if (
#             len(patch_size_per_conv_layer) != n_conv_layers
#             or len(stride_per_conv_layer) != n_conv_layers
#         ):
#             raise Exception(
#                 "The lengths of n_hiddens_per_conv_layer, patch_size_per_conv_layer, and stride_per_conv_layer must be equal."
#             )

#         self.activation_function = (
#             torch.tanh if activation_function == "tanh" else torch.relu
#         )

#         # Create all convolutional layers
#         # First argument to first Conv1d is number of channels for a sample
#         n_in = 14
#         input_hw = int(
#             np.sqrt(n_inputs)
#         )  # original input image height (=width because image assumed square)
#         self.conv_layers = torch.nn.ModuleList()
#         for nh, patch_size, stride in zip(
#             n_hiddens_per_conv_layer, patch_size_per_conv_layer, stride_per_conv_layer
#         ):
#             self.conv_layers.append(
#                 torch.nn.Conv1d(n_in, nh, kernel_size=patch_size, stride=stride)
#             )
#             conv_layer_output_hw = (input_hw - patch_size) // stride + 1
#             input_hw = conv_layer_output_hw  # for next trip through this loop
#             n_in = nh

#         # Create all fully connected layers.  First must determine number of inputs to first
#         # fully-connected layer that results from flattening the images coming out of the last
#         # convolutional layer.
#         n_in = input_hw ** 2 * n_in  # n_hiddens_per_fc_layer[0]
#         self.fc_layers = torch.nn.ModuleList()
#         for nh in n_hiddens_per_fc_layer:
#             self.fc_layers.append(torch.nn.Linear(n_in, nh))
#             n_in = nh
#         self.fc_layers.append(torch.nn.Linear(n_in, n_outputs))

#         self.to(self.device)

#     def forward_all_outputs(self, X):
#         n_samples = X.shape[0]
#         Ys = [X]
#         for conv_layer in self.conv_layers:
#             Ys.append(self.activation_function(conv_layer(Ys[-1])))

#         for layeri, fc_layer in enumerate(self.fc_layers[:-1]):
#             if layeri == 0:
#                 Ys.append(
#                     self.activation_function(fc_layer(Ys[-1].reshape(n_samples, -1)))
#                 )
#             else:
#                 Ys.append(self.activation_function(fc_layer(Ys[-1])))

#         Ys.append(self.fc_layers[-1](Ys[-1]))
#         return Ys

#     def forward(self, X):
#         Ys = self.forward_all_outputs(X)
#         return Ys[-1]

#     def train(
#         self, X, T, batch_size, n_epochs, learning_rate, method="sgd", verbose=True
#     ):

#         # Set data matrices to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         if not isinstance(T, torch.Tensor):
#             T = (
#                 torch.from_numpy(T).long().to(self.device)
#             )  # required for classification in pytorch

#         X.requires_grad_(True)

#         self.classes = torch.unique(T)

#         if method == "sgd":
#             optimizer = torch.optim.SGD(
#                 self.parameters(), lr=learning_rate, momentum=0.9
#             )
#         else:
#             optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

#         CELoss = torch.nn.CrossEntropyLoss(reduction="mean")
#         self.error_trace = []

#         for epoch in range(n_epochs):

#             num_batches = X.shape[0] // batch_size
#             loss_sum = 0

#             for k in range(num_batches):
#                 start = k * batch_size
#                 end = (k + 1) * batch_size
#                 X_batch = X[start:end, ...]
#                 T_batch = T[start:end, ...]

#                 Y = self.forward(X_batch)

#                 loss = CELoss(Y, T_batch)
#                 loss.backward()

#                 # Update parameters
#                 optimizer.step()
#                 optimizer.zero_grad()

#                 loss_sum += loss

#             self.error_trace.append(loss_sum / num_batches)

#             if n_epochs >= 10:
#                 if verbose and (epoch + 1) % (n_epochs // 10) == 0:
#                     print(
#                         f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}"
#                     )
#             else:
#                 print(f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}")

#         return self

#     def softmax(self, Y):
#         """Apply to final layer weighted sum outputs"""
#         # Trick to avoid overflow
#         maxY = torch.max(Y, axis=1)[0].reshape((-1, 1))
#         expY = torch.exp(Y - maxY)
#         denom = torch.sum(expY, axis=1).reshape((-1, 1))
#         Y = expY / denom
#         return Y

#     def use(self, X):
#         # Set input matrix to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         Y = self.forward(X)
#         probs = self.softmax(Y)
#         classes = self.classes[torch.argmax(probs, axis=1).cpu().numpy()]
#         return classes.cpu().numpy(), probs.detach().cpu().numpy()

In [8]:
# import random


# with open("Dummy_Lumped_Test.csv", "w", newline="") as csvfile:
#     spamwriter = csv.writer(
#         csvfile, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
#     )
#     col_labels = [
#         "AF3_0",
#         "F7_0",
#         "F3_0",
#         "FC5_0",
#         "T7_0",
#         "P7_0",
#         "O1_0",
#         "O2_0",
#         "P8_0",
#         "T8_0",
#         "FC6_0",
#         "F4_0",
#         "F8_0",
#         "AF4_0",
#         "AF3_1",
#         "F7_1",
#         "F3_1",
#         "FC5_1",
#         "T7_1",
#         "P7_1",
#         "O1_1",
#         "O2_1",
#         "P8_1",
#         "T8_1",
#         "FC6_1",
#         "F4_1",
#         "F8_1",
#         "AF4_1",
#         "AF3_2",
#         "F7_2",
#         "F3_2",
#         "FC5_2",
#         "T7_2",
#         "P7_2",
#         "O1_2",
#         "O2_2",
#         "P8_2",
#         "T8_2",
#         "FC6_2",
#         "F4_2",
#         "F8_2",
#         "AF4_2",
#         "AF3_3",
#         "F7_3",
#         "F3_3",
#         "FC5_3",
#         "T7_3",
#         "P7_3",
#         "O1_3",
#         "O2_3",
#         "P8_3",
#         "T8_3",
#         "FC6_3",
#         "F4_3",
#         "F8_3",
#         "AF4_3",
#         "AF3_4",
#         "F7_4",
#         "F3_4",
#         "FC5_4",
#         "T7_4",
#         "P7_4",
#         "O1_4",
#         "O2_4",
#         "P8_4",
#         "T8_4",
#         "FC6_4",
#         "F4_4",
#         "F8_4",
#         "AF4_4",
#         "AF3_5",
#         "F7_5",
#         "F3_5",
#         "FC5_5",
#         "T7_5",
#         "P7_5",
#         "O1_5",
#         "O2_5",
#         "P8_5",
#         "T8_5",
#         "FC6_5",
#         "F4_5",
#         "F8_5",
#         "AF4_5",
#         "AF3_6",
#         "F7_6",
#         "F3_6",
#         "FC5_6",
#         "T7_6",
#         "P7_6",
#         "O1_6",
#         "O2_6",
#         "P8_6",
#         "T8_6",
#         "FC6_6",
#         "F4_6",
#         "F8_6",
#         "AF4_6",
#         "AF3_7",
#         "F7_7",
#         "F3_7",
#         "FC5_7",
#         "T7_7",
#         "P7_7",
#         "O1_7",
#         "O2_7",
#         "P8_7",
#         "T8_7",
#         "FC6_7",
#         "F4_7",
#         "F8_7",
#         "AF4_7",
#         "AF3_8",
#         "F7_8",
#         "F3_8",
#         "FC5_8",
#         "T7_8",
#         "P7_8",
#         "O1_8",
#         "O2_8",
#         "P8_8",
#         "T8_8",
#         "FC6_8",
#         "F4_8",
#         "F8_8",
#         "AF4_8",
#         "AF3_9",
#         "F7_9",
#         "F3_9",
#         "FC5_9",
#         "T7_9",
#         "P7_9",
#         "O1_9",
#         "O2_9",
#         "P8_9",
#         "T8_9",
#         "FC6_9",
#         "F4_9",
#         "F8_9",
#         "AF4_9",
#         "eyeDetection",
#     ]
#     spamwriter.writerow(col_labels)

#     list_a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
#     list_b = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
#     list_big_a = []
#     list_big_b = []
#     for i in range(10):
#         for j in range(14):
#             list_big_a.append(list_a[j])
#             list_big_b.append(list_b[j])

#     list_big_a.append(0)
#     list_big_b.append(1)
#     for i in range(500):
#         random_number = random.randint(1, 10)
#         if random_number > 5:
#             spamwriter.writerow(list_big_a)
#         else:
#             spamwriter.writerow(list_big_b)

In [9]:
model = fpnn.CNN(
    1,
    14,
    [10, 20],
    [5, 5],
    2,
    kernel_size_per_conv_layer=[3, 3],
    stride_per_conv_layer=[2, 2],
).to(device)
print(model)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(3,), stride=(2,))
    (1): Conv1d(10, 20, kernel_size=(3,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=40, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=5, bias=True)
    (2): Linear(in_features=5, out_features=2, bias=True)
  )
)


In [10]:
# from functools import reduce
# from operator import mul

# print(torch.rand(5, 14))
# X = torch.rand(100, 14, device=device)
# print(X.shape)
# T = torch.randint(0, 2, (100, 1), device=device)
# logits = model(X)
# print(logits)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    # first = True
    for batch, (X, T) in enumerate(dataloader):
        X, T = X.to(device), T.to(device)

        # Compute prediction error
        Y = model(X.float())
        # if first:
        #     first = False
        #     print('Y.shape', Y.shape)
        #     print('T.shape', T.shape)
        #     print('T: ', T[0])
        loss = loss_fn(Y, T.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


#         if batch % 50 == 0:
#             loss, current = loss.item(), batch * len(X)
# print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, T in dataloader:
            X, T = X.to(device), T.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, T.long()).item()
            correct += (pred.argmax(1) == T).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [13]:
def train_for_epochs(
    epochs, train_dataloader, _validate_dataloader, model, loss_fn, optimizer
):
    start = time.time()
    for t in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        if t % 50 == 0:
            print(f"Epoch {t}\n-------------------------------")
            test(_validate_dataloader, model, loss_fn)
    end = time.time()
    print("Done: ", end - start, " seconds")
    test(_validate_dataloader, model, loss_fn)

In [14]:
# fashion_training_data = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )
# fashion_train_dataloader = DataLoader(fashion_training_data, batch_size=64)
# n_samples, n_inputs = train_dataset.__len__(), 28*28
# n_outputs = 1
# n_hiddens = [10, 10]
# model = fpnn.NeuralNetwork(28*28, n_hiddens, 10).to(device)
# print(model)

# learning_rate = 0.5 / (n_samples * n_outputs)  ## Larger learning rate

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# train(fashion_train_dataloader, model, loss_fn, optimizer)

In [15]:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 14])
Shape of y:  torch.Size([64]) torch.float64


I again started out with a dummy run using dummy data that had a trivial pattern which the neural network should easily be able to learn. I did this to simply test that the network was correctly setup.

In [22]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

# train_file = "EEG_Eye_State_Train.csv"
# validate_file = "EEG_Eye_State_Validate.csv"
# test_file = "EEG_Eye_State_Test.csv"

train_file = "Dummy_Lumped_Train.csv"
validate_file = "Dummy_Lumped_Validate.csv"
test_file = "Dummy_Lumped_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [23]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 100
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.069871 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000094 

Done:  11.338661193847656  seconds
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000063 



In [24]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000063 



domain model result.

In [25]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Lumped_10_Normalized_Train.csv"
validate_file = "EEG_Eye_State_Lumped_10_Normalized_Validate.csv"
test_file = "EEG_Eye_State_Lumped_10_Normalized_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [27]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.603634 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.299650 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.278495 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.282074 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.288265 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.308168 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg

In [28]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.631273 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.207903 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.228532 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.238833 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.245316 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.250075 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg

In [30]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [1000, 20, 10]
num_hiddens_per_fc_layer = [500, 15, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10]
stride_per_conv_layer = [1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 1000, kernel_size=(10,), stride=(1,))
    (1): Conv1d(1000, 20, kernel_size=(10,), stride=(1,))
    (2): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1130, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.608456 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.269878 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.382582 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.410375 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.427347 

Epoch 250
-------------------------------
Test E

In [31]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [10]
num_hiddens_per_fc_layer = [10]
num_outputs = 2
kernel_size_per_conv_layer = [10]
stride_per_conv_layer = [1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1310, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 63.8%, Avg loss: 0.554793 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.407030 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.443656 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.463671 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.470202 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.474084 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.476955 

Epoch 350
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.479734 

Epoc

In [32]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [
    int(np.sqrt(np.prod(117))) * 2,
    int(np.sqrt(np.prod(117))),
    10,
]
num_hiddens_per_fc_layer = [
    int(np.sqrt(np.prod(117))) * 2,
    int(np.sqrt(np.prod(117))),
    10,
]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10]
stride_per_conv_layer = [1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
    (2): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1130, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.707113 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.275624 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.357023 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.402117 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.421686 

Epoch 250
-------------------------------
Test Error: 

In [34]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20, 10, 10, 5]
num_hiddens_per_fc_layer = [20, 15, 15, 10, 5]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10, 10, 10]
stride_per_conv_layer = [1, 1, 1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
    (2): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
    (3): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
    (4): Conv1d(10, 5, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=475, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=15, bias=True)
    (3): Linear(in_features=15, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=5, bias=True)
    (5): Linear(in_features=5, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.699657 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.269158 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.507028 

Epoch 150
----

In [35]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [100, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 100, kernel_size=(10,), stride=(1,))
    (1): Conv1d(100, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.0%, Avg loss: 0.596524 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.268456 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.331914 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.362615 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.382574 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.395071 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, A

In [ ]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [100, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 100, kernel_size=(10,), stride=(1,))
    (1): Conv1d(100, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.0%, Avg loss: 0.596524 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.268456 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.331914 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.362615 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.382574 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.395071 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, A

In [ ]:
test(train_dataloader, model, loss_fn)

20